In [1]:
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import finpy_tse as fpy
import pytse_client as tse
import yfinance as yf
import ta 
import requests
import asyncio
import math as math

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [3]:
last_day =datetime.now()
last_day=last_day.date()
last_day=str(last_day)
last_day

'2024-05-24'

In [4]:
count=750   # tedad candle data

In [5]:
forecast=5          #  تعداد روز برای پیشبینی

In [6]:


#fpy>>>>> index tepix

def index_kol(bars:int=9500):
        while True:
            try:
                data=fpy.Get_CWI_History(ignore_date=True,double_date=True)
                data=data.iloc[:-bars:-1][::-1]
                data.index=data['Date']
                header = {'User-Agent': "ADN"}
                adres = 'http://cdn.tsetmc.com/api/Index/GetIndexB1LastDay/32097828799138957'
                darkhast = requests.get(url=adres, headers=header).text.split('},{')

                for a in darkhast:
                    close_tepix = int(a.split('xDrNivJIdx004":')[1].split('.')[0])
                    high_tepix = int(a.split('xPhNivJIdx004":')[1].split('.')[0])
                    low_tepix = int(a.split('xPbNivJIdx004":')[1].split('.')[0])
                    open_tepix = int(darkhast[7].split('xDrNivJIdx004":')[1].split('.')[0])
                    
                formatted_date = pd.to_datetime(last_day)
                # اطلاعات خود را به صورت یک دیکشنری تعریف می‌کنیم
                data_tepix = {
                    'Date':[formatted_date],
                    'Open':[open_tepix],
                    'High':[high_tepix],
                    'Low':[low_tepix],
                    'Close':[close_tepix],
                    'Adj Close':[close_tepix],
                    'Volume':0,  }
                # دیکشنری را به DataFrame تبدیل می‌کنیم
                instant_tepix = pd.DataFrame(data_tepix)
                instant_tepix.index=instant_tepix['Date']
                instant_tepix = pd.DataFrame(instant_tepix)
                instant_tepix = pd.concat([instant_tepix, data])
                instant_tepix = instant_tepix.sort_index(ascending=True)
                
                if  (instant_tepix.iloc[-1]['Close'] == instant_tepix.iloc[-2]['Close']  )   or  (  abs(instant_tepix.iloc[-1]['Close'] - instant_tepix.iloc[-2]['Close']  ) < 2   or  (instant_tepix.index[-1]== instant_tepix.index[-2] )):
                    instant_tepix = instant_tepix.iloc[:-1]
                return(instant_tepix)
            except Exception as e:
                print(f"An error occurred: {e}")
                asyncio.sleep(2)  # Sleep for 10 seconds before retrying



symbol_data=index_kol()
symbol_data

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2008-12-06,2008-12-06,9249.0,9249.0,9168.0,9178.0,9178.3,8539624
2008-12-07,2008-12-07,9178.0,9178.0,9130.0,9130.0,9130.5,11752353
2008-12-08,2008-12-08,9103.0,9103.0,9080.0,9089.0,9089.2,15299115
2008-12-10,2008-12-10,9072.0,9072.0,9024.0,9024.0,9023.7,15689653
2008-12-13,2008-12-13,8973.0,8973.0,8973.0,8973.0,8973.3,31428174
...,...,...,...,...,...,...,...
2024-05-15,2024-05-15,2165990.0,2166380.0,2159160.0,2164692.0,2164692.3,4125186697
2024-05-18,2024-05-18,2166800.0,2174950.0,2166800.0,2168676.0,2168676.3,4410308923
2024-05-19,2024-05-19,2166180.0,2166180.0,2106430.0,2112560.0,2112560.3,5689636315


In [7]:
symbol_data=symbol_data.tail(count)
symbol_data

,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2021-04-07,2021-04-07,1258600.0,1258600.0,1249890.0,1249898.0,1249910.4,1068639226
2021-04-10,2021-04-10,1246740.0,1246740.0,1240750.0,1240759.0,1240696.7,588396551
2021-04-11,2021-04-11,1237340.0,1237340.0,1230920.0,1230927.0,1230876.5,1265213974
2021-04-12,2021-04-12,1228760.0,1228760.0,1225810.0,1226119.0,1226158.2,2409026367
2021-04-13,2021-04-13,1234640.0,1245540.0,1234640.0,1244587.0,1244556.7,5631017338
...,...,...,...,...,...,...,...
2024-05-15,2024-05-15,2165990.0,2166380.0,2159160.0,2164692.0,2164692.3,4125186697
2024-05-18,2024-05-18,2166800.0,2174950.0,2166800.0,2168676.0,2168676.3,4410308923
2024-05-19,2024-05-19,2166180.0,2166180.0,2106430.0,2112560.0,2112560.3,5689636315


In [8]:
df=symbol_data[['Close']]
df

,Close
Date,
2021-04-07,1249898.0
2021-04-10,1240759.0
2021-04-11,1230927.0
2021-04-12,1226119.0
2021-04-13,1244587.0
...,...
2024-05-15,2164692.0
2024-05-18,2168676.0
2024-05-19,2112560.0


In [9]:

df['Prediction']=df[['Close']].shift(-forecast)
df

C:\Users\MSI\AppData\Local\Temp\ipykernel_4812\598704838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Prediction']=df[['Close']].shift(-forecast)


,Close,Prediction
Date,,
2021-04-07,1249898.0,1243236.0
2021-04-10,1240759.0,1233724.0
2021-04-11,1230927.0,1224778.0
2021-04-12,1226119.0,1216836.0
2021-04-13,1244587.0,1219665.0
...,...,...
2024-05-15,2164692.0,NaN
2024-05-18,2168676.0,NaN
2024-05-19,2112560.0,NaN


In [10]:

x = np.array(df.drop(['Prediction'], axis=1))
x= x[:-forecast]
y= np.array(df['Prediction'])
y=y[:-forecast]

In [11]:

xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2)
mysvr=SVR(kernel='rbf',C=1000,gamma=0.1)
mysvr.fit(xtrain,ytrain)
svconf=mysvr.score(xtrain,ytrain)
print('svm confidence: ',svconf)    #  الگوریتم ماشین بردار


svm confidence:  -0.17145291468839896


In [12]:
lr=LinearRegression()
lr.fit(xtrain,ytrain)
lrconf=lr.score(xtest,ytest)
print('LR confidence: ',lrconf)    # الگوریتم رگرسیون خطی

LR confidence:  0.9780506450947971


In [13]:

x_forecast=np.array(df.drop(['Prediction'],axis=1))[-forecast:]
lrpred= lr.predict(x_forecast)
print('LR Method: ' ,lrpred)
print('-----------')
svmpred=mysvr.predict(x_forecast)
print('svm Method:' ,svmpred)

LR Method:  [2162713.67406993 2166632.49273453 2111434.59409432 2111434.59409432
 2112992.67862362]
-----------
svm Method: [1522443.4 1522443.4 1522443.4 1522443.4 1522443.4]


In [14]:
lrpred=pd.DataFrame(lrpred)

In [15]:
lrpred_names = ['Prediction']  # جایگزین کردن با نام‌های دلخواهتان

# تغییر نام ستون‌ها در دیتافریم
lrpred.columns = lrpred_names

In [16]:
lrpred=round(lrpred)
lrpred=lrpred.astype(int)
lrpred

,Prediction
0,2162714
1,2166632
2,2111435
3,2111435
4,2112993
